In [1]:
using CSV
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-01-23_one_column.csv"))

,Column1,hospital_name,ccn,collection_week,fips_code,hospitalization_percentage,ICU_percentages,hospitalization_2020-12-29,target_date
,Int64,String,String?,Date,Float64,Float64?,Float64?,Float64?,Date
1,10,PRATTVILLE BAPTIST HOSPITAL,010108,2020-12-25,1001.0,1.0,0.0944056,3.53556,2020-12-29
2,11,NORTH BALDWIN INFIRMARY,010129,2020-12-25,1003.0,0.164736,0.0795455,1.76119,2020-12-29
3,12,THOMAS HOSPITAL,010100,2020-12-25,1003.0,0.520986,0.195402,5.56984,2020-12-29
4,13,SOUTH BALDWIN REGIONAL MEDICAL CENTER,010083,2020-12-25,1003.0,0.314278,0.441718,3.35993,2020-12-29
5,14,MEDICAL CENTER BARBOUR,010069,2020-12-25,1005.0,1.0,0.328125,0.94236,2020-12-29
6,15,BIBB MEDICAL CENTER,010058,2020-12-25,1007.0,0.863636,missing,1.39368,2020-12-29
7,16,ST VINCENTS BLOUNT,011305,2020-12-25,1009.0,0.954545,0.238636,3.25057,2020-12-29
8,17,BULLOCK COUNTY HOSPITAL,010110,2020-12-25,1011.0,0.0,missing,0.0,2020-12-29
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,010150,2020-12-25,1013.0,0.954545,0.32,0.662337,2020-12-29


In [4]:
data = select(rawdata,
    :hospital_name => :hospital,
    :ccn => :hospital_id,
    :target_date => :date,
    ["hospitalization_2020-12-29", "ICU_percentages"] => ByRow((x,y) -> x*coalesce(y,0.3)) => :admitted_moderate_icu,
    ["hospitalization_2020-12-29", "ICU_percentages"] => ByRow((x,y) -> x*(1.0-coalesce(y,0.3))) => :admitted_moderate_acute,
    "hospitalization_2020-12-29" => :admitted_moderate_allbeds,
)
sort!(data, [:hospital, :hospital_id, :date])

,hospital,hospital_id,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,String?,Date,Float64?,Float64?,Float64?
1,ABBEVILLE AREA MEDICAL CENTER,421301,2020-12-29,0.0,0.554505,0.554505
2,ABBEVILLE AREA MEDICAL CENTER,missing,2020-12-30,0.0,0.573475,0.573475
3,ABBEVILLE AREA MEDICAL CENTER,missing,2020-12-31,0.0,0.612186,0.612186
4,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-01,0.0,0.603726,0.603726
5,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-02,0.0,0.617056,0.617056
6,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-03,0.0,0.581469,0.581469
7,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-04,0.0,0.55883,0.55883
8,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-05,0.0,0.555632,0.555632
9,ABBEVILLE AREA MEDICAL CENTER,missing,2021-01-06,0.0,0.566951,0.566951


In [5]:
data |> CSV.write("../data/hhs_forecast_2021_01_08.csv")

"../data/hhs_forecast_2021_01_08.csv"